In [15]:
!conda install -c conda-forge beautifulsoup4 --yes

!conda install -c conda-forge geopy --yes

!conda install -c conda-forge folium=0.5.0 --yes

print('Libraries installed!')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries installed!


In [16]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
from pandas.io.json import json_normalize
import json

import requests

from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim

import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

print('Libraries imported!')

Libraries imported!


In [17]:
import urllib.request # open and read URLs

import json # handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import requests # handle requests
import pandas as pd # process data as dataframes with Pandas
import numpy as np # handle data in a vectorized manner with NumPy

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't installed the GeoPy geocoding library yet
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't installed the Folium library yet
import folium # map rendering library

# Matplolib plotting library and associated modules
import matplotlib.pyplot as plt 
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans # for K-Means clustering with Scikit-Learn

print("Libraries imported!")

Libraries imported!


In [ ]:
# Get data about Paris geography from https://opendata.paris.fr website

with urllib.request.urlopen("https://opendata.paris.fr/explore/dataset/arrondissements/download?format=json&timezone=Europe/Berlin&use_labels_for_header=true") as url:
   data = json.loads(url.read())
data[0]

In [ ]:
# Tranform the json data into a Pandas dataframe

par_nei = json_normalize(data)
par_nei.head()

In [ ]:
# Format the dataframe

df_temp1 = par_nei[["fields.c_ar","fields.l_aroff","fields.geom_x_y"]]
df_temp2 = pd.DataFrame(par_nei['fields.geom_x_y'].tolist(), columns=["Latitude", "Longitude"])
df_drop = df_temp1.drop(["fields.geom_x_y"], axis = 1)
df_temp = pd.concat([df_drop,df_temp2], axis=1)
df_temp.columns = ["Arrondissement", "Neighborhood", "Latitude", "Longitude"]
paris_data = df_temp.sort_values(["Arrondissement"]).reset_index(drop=True)

# change neighborhood name Élysée to Elysée
paris_data.loc[paris_data.Neighborhood == "Élysée", ["Neighborhood"]] = "Elysée"

paris_data

In [ ]:
# Use GeoPy library to get the latitude and longitude values of Paris

address = "Paris, FR"

geolocator = Nominatim(user_agent="my-application")
location_par = geolocator.geocode(address)
latitude_par = location_par.latitude
longitude_par = location_par.longitude

print("The geograpical coordinates of Paris are {}, {}.".format(latitude_par, longitude_par))

In [ ]:
# Create a leaflet map of Paris with arrondissements superimposed on top

# create a map of Paris using latitude and longitude values
map_paris = folium.Map(location=[latitude_par, longitude_par], zoom_start=12)

# add markers to the map
for lat, lng, arrondissement, neighborhood in zip(paris_data["Latitude"], paris_data["Longitude"], paris_data["Arrondissement"], paris_data["Neighborhood"]):
    label = "{}, {}".format(arrondissement,neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = "blue",
        fill = True,
        fill_color = "#3186cc",
        fill_opacity = 0.7,
        parse_html = False).add_to(map_paris)  
    
map_paris

In [ ]:
# Get data about New York geography from https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json

with urllib.request.urlopen("https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json") as file:
   newyork_data = json.loads(file.read())
#newyork_data

In [ ]:
# Tranform the json data into a formatted Pandas dataframe

# all the relevant data is in the features key, which is basically a list of the neighborhoods
neighborhoods_data = newyork_data["features"]

# define the dataframe columns
column_names = ["Borough", "Neighborhood", "Latitude", "Longitude"] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

# loop through the data and fill the dataframe one row at a time
for data in neighborhoods_data:
    borough = neighborhood_name = data["properties"]["borough"] 
    neighborhood_name = data["properties"]["name"]
        
    neighborhood_latlon = data["geometry"]["coordinates"]
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({"Borough": borough,
                                          "Neighborhood": neighborhood_name,
                                          "Latitude": neighborhood_lat,
                                          "Longitude": neighborhood_lon}, ignore_index=True)
    
neighborhoods.head()

In [ ]:
manhattan_data = neighborhoods[neighborhoods["Borough"] == "Manhattan"].reset_index(drop=True)
manhattan_data

In [ ]:
address = "Manhattan, NY"

geolocator = Nominatim(user_agent="my-application")
location_man = geolocator.geocode(address)
latitude_man = location_man.latitude
longitude_man = location_man.longitude

print("The geograpical coordinates of Manhattan are {}, {}.".format(latitude_man, longitude_man))

In [ ]:
# Create a leaflet map of Manhattan with neighborhoods superimposed on top

# create a map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude_man, longitude_man], zoom_start=11)

# add markers to the map
for lat, lng, neighborhood in zip(manhattan_data["Latitude"], manhattan_data["Longitude"], manhattan_data["Neighborhood"]):
    label = "{}".format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = "red",
        fill = True,
        fill_color = "#E03450",
        fill_opacity = 0.7,
        parse_html = False).add_to(map_manhattan)  
    
map_manhattan

In [ ]:
CLIENT_ID = "MF4VNN5LCLCUG1F01NH2JR2SJ0ILYOGLVCDHD2KGNXIBTT1U" 
CLIENT_SECRET = "GQ04P00OKBMLX0EUM0FGO5JOMA44JFSAEUOTFYGZY35XOSSC" 
VERSION = "20180605"

In [ ]:
# Define a function to gather recommended venues, with specifically the name and category, using the explore API

def getNearbyVenues_paris(arrondissements, names, latitudes, longitudes, radius=750, limit=100):
    
    venues_list = []
    for arrondissement, name, lat, lng in zip(arrondissements, names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            arrondissement,
            name, 
            lat, 
            lng, 
            v["venue"]["name"], 
            v["venue"]["location"]["lat"], 
            v["venue"]["location"]["lng"],  
               v["venue"]["categories"][0]["name"]) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ["Arrondissement",
                  "Neighborhood", 
                  "Neighborhood Latitude", 
                  "Neighborhood Longitude", 
                  "Venue", 
                  "Venue Latitude", 
                  "Venue Longitude", 
                  "Venue Category"]
    
    return(nearby_venues)

In [ ]:
# Create a new dataframe called paris_venues that combines the recommended venues from Foursquare 
# and neigborhoods geo-information from the city dataset

paris_venues = getNearbyVenues_paris(arrondissements=paris_data["Arrondissement"],
                               names = paris_data["Neighborhood"],
                               latitudes = paris_data["Latitude"],
                               longitudes = paris_data["Longitude"]
                               )

In [ ]:
print(paris_venues.shape)
print("There are {} unique categories.".format(len(paris_venues["Venue Category"].unique())))
paris_venues.head()


In [ ]:
# Define a function to gather recommended venues, with specifically the name and category, using the explore API

def getNearbyVenues_manhattan(boroughs, names, latitudes, longitudes, radius=750, limit=100):
    
    venues_list = []
    for borough, name, lat, lng in zip(boroughs, names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            borough,
            name, 
            lat, 
            lng, 
            v["venue"]["name"], 
            v["venue"]["location"]["lat"], 
            v["venue"]["location"]["lng"],  
            v["venue"]["categories"][0]["name"]) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ["Borough",
                  "Neighborhood", 
                  "Neighborhood Latitude", 
                  "Neighborhood Longitude", 
                  "Venue", 
                  "Venue Latitude", 
                  "Venue Longitude", 
                  "Venue Category"]
    
    return(nearby_venues)

In [ ]:
# Create a new dataframe called manhattan_venues that combines the recommended venues from Foursquare 
# and neigborhoods geo-information from the city dataset

manhattan_venues = getNearbyVenues_manhattan(boroughs=manhattan_data["Borough"],
                               names = manhattan_data["Neighborhood"],
                               latitudes = manhattan_data["Latitude"],
                               longitudes = manhattan_data["Longitude"]
                               )

In [ ]:
print(manhattan_venues.shape)
print("There are {} unique categories.".format(len(manhattan_venues["Venue Category"].unique())))
manhattan_venues.head()

In [ ]:
paris_onehot = pd.get_dummies(paris_venues[["Venue Category"]], prefix="", prefix_sep="")

# add arrondissement and neighborhood column back to dataframe
paris_onehot["Arrondissement"] = paris_venues["Arrondissement"] 
paris_onehot["Neighborhood"] = paris_venues["Neighborhood"] 

# move arrondissement and neighborhood columns to the first columns
fixed_columns = [paris_onehot.columns[-2], paris_onehot.columns[-1]] + list(paris_onehot.columns[:-2])
paris_onehot = paris_onehot[fixed_columns]

paris_onehot.head()

In [ ]:
# Group by neighborhood and by the mean of frequency for each category

paris_grouped = paris_onehot.groupby("Neighborhood").mean().reset_index()
paris_grouped.head()

In [ ]:
# Top 5 most common categories of venues for each neighborhood 

num_top_venues = 5

for hood in paris_grouped["Neighborhood"]:
    print("----"+hood+"----")
    temp = paris_grouped[paris_grouped["Neighborhood"]==hood].T.reset_index()
    temp.columns = ["venue","freq"]
    temp = temp.iloc[2:]
    temp["freq"] = temp["freq"].astype(float)
    temp = temp.round({"freq": 2})
    print(temp.sort_values("freq", ascending=False).reset_index(drop=True).head(num_top_venues))
    print("\n")

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
# Create a new dataframe and display the top 5 venues for each neighborhood

num_top_venues = 10

indicators = ["st", "nd", "rd"]

# create columns according to number of top venues
columns = ["Arrondissement", "Neighborhood"]
for ind in np.arange(num_top_venues):
    try:
        columns.append("{}{} Most Common Venue".format(ind+1, indicators[ind]))
    except:
        columns.append("{}th Most Common Venue".format(ind+1))

# create a new dataframe
paris_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
paris_neighborhoods_venues_sorted[[ "Arrondissement","Neighborhood"]] = paris_grouped[["Arrondissement","Neighborhood"]]

for ind in np.arange(paris_grouped.shape[0]):
    paris_neighborhoods_venues_sorted.iloc[ind, 2:] = return_most_common_venues(paris_grouped.iloc[ind, 1:], num_top_venues)

paris_neighborhoods_venues_sorted

In [ ]:
# one hot encoding

manhattan_onehot = pd.get_dummies(manhattan_venues[["Venue Category"]], prefix="", prefix_sep="")

# add arrondissement and neighborhood column back to dataframe
manhattan_onehot["Neighborhood"] = manhattan_venues["Neighborhood"] 

# move arrondissement and neighborhood columns to the first columns
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

In [ ]:
# Group by neighborhood and by the mean of frequency for each category

manhattan_grouped = manhattan_onehot.groupby("Neighborhood").mean().reset_index()
manhattan_grouped.head()

In [ ]:
num_top_venues = 5

for hood in manhattan_grouped["Neighborhood"]:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped["Neighborhood"]==hood].T.reset_index()
    temp.columns = ["venue","freq"]
    temp = temp.iloc[1:]
    temp["freq"] = temp["freq"].astype(float)
    temp = temp.round({"freq": 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print("\n")

In [ ]:
num_top_venues = 10

indicators = ["st", "nd", "rd"]

# create columns according to number of top venues
columns = ["Neighborhood"]
for ind in np.arange(num_top_venues):
    try:
        columns.append("{}{} Most Common Venue".format(ind+1, indicators[ind]))
    except:
        columns.append("{}th Most Common Venue".format(ind+1))

# create a new dataframe
manhattan_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
manhattan_neighborhoods_venues_sorted[["Neighborhood"]] = manhattan_grouped[["Neighborhood"]]

for ind in np.arange(manhattan_grouped.shape[0]):
    manhattan_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

manhattan_neighborhoods_venues_sorted

In [ ]:
# Create the metric (same squared distances) to identify the best k

columns = ["Neighborhood", "Arrondissement"]
paris_grouped_clustering = paris_grouped.drop(columns, axis = 1)

Sum_of_squared_distances = []

ks = range(1,11)
for k in ks:
    kmeans = KMeans(n_clusters=k, random_state=0).fit(paris_grouped_clustering)
    Sum_of_squared_distances.append(kmeans.inertia_)

In [ ]:
# Plot of sum of squared distances

plt.plot(ks, Sum_of_squared_distances, "bx-")
plt.xlabel("k number of clusters")
plt.ylabel("Sum_of_squared_distances")
plt.title("Elbow method for optimal k")
plt.show()


In [ ]:
# Cluster the venues of Paris into 5 clusters

# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(paris_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
paris_merged = paris_data
 
# add clustering labels
paris_merged["Cluster Labels"] = kmeans.labels_
paris_merged = pd.merge(paris_merged, paris_neighborhoods_venues_sorted,  how = "left", left_on = ["Arrondissement","Neighborhood"], right_on = ["Arrondissement","Neighborhood"])

paris_merged


In [ ]:
# create map
map_clusters = folium.Map(location=[latitude_par, longitude_par], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(paris_merged["Latitude"], paris_merged["Longitude"], paris_merged["Neighborhood"], paris_merged["Cluster Labels"]):
    label = folium.Popup(str(poi) + "Cluster" + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster-1],
        fill = True,
        fill_color = rainbow[cluster-1],
        fill_opacity = 0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
# Create the metric (same squared distances) to identify the best k

columns = ["Neighborhood"]
manhattan_grouped_clustering = manhattan_grouped.drop(columns, axis = 1)

Sum_of_squared_distances = []

ks = range(1,11)
for k in ks:
    kmeans = KMeans(n_clusters=k, random_state=0).fit(manhattan_grouped_clustering)
    Sum_of_squared_distances.append(kmeans.inertia_)


In [ ]:
# Plot of sum of squared distances

plt.plot(ks, Sum_of_squared_distances, "bx-")
plt.xlabel("k number of clusters")
plt.ylabel("Sum_of_squared_distances")
plt.title("Elbow method for optimal k")
plt.show()

In [ ]:
# set number of clusters
kclusters = 8

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
manhattan_merged = manhattan_data

# add clustering labels
manhattan_merged["Cluster Labels"] = kmeans.labels_

# merge manhattan_grouped with manhattann_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(manhattan_neighborhoods_venues_sorted.set_index("Neighborhood"), on="Neighborhood")

manhattan_merged # check the last columns!

In [ ]:
map_clusters = folium.Map(location=[latitude_man, longitude_man], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged["Latitude"], manhattan_merged["Longitude"], manhattan_merged["Neighborhood"], manhattan_merged["Cluster Labels"]):
    label = folium.Popup(str(poi) + "Cluster" + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster-1],
        fill = True,
        fill_color = rainbow[cluster-1],
        fill_opacity = 0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
# Cluster 1

paris_merged.loc[paris_merged["Cluster Labels"] == 0, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

In [ ]:
# Cluster 2

paris_merged.loc[paris_merged["Cluster Labels"] == 1, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

In [ ]:
# Cluster 3

paris_merged.loc[paris_merged["Cluster Labels"] == 2, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

In [ ]:
# Cluster 4

paris_merged.loc[paris_merged["Cluster Labels"] == 3, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

In [ ]:
# Cluster 5

paris_merged.loc[paris_merged["Cluster Labels"] == 4, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

In [ ]:
# Cluster 1

manhattan_merged.loc[manhattan_merged["Cluster Labels"] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]


In [ ]:
# Cluster 2

manhattan_merged.loc[manhattan_merged["Cluster Labels"] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

In [ ]:
# Cluster 3

manhattan_merged.loc[manhattan_merged["Cluster Labels"] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

In [ ]:
# Cluster 4

manhattan_merged.loc[manhattan_merged["Cluster Labels"] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]


In [ ]:
manhattan_merged.loc[manhattan_merged["Cluster Labels"] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

In [ ]:
# Cluster 6

manhattan_merged.loc[manhattan_merged["Cluster Labels"] == 5, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]


In [ ]:
manhattan_merged.loc[manhattan_merged["Cluster Labels"] == 6, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

In [ ]:
# Cluster 8

manhattan_merged.loc[manhattan_merged["Cluster Labels"] == 7, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]
